## Tacotron 2 inference code 
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

#### Import libraries and setup matplotlib

In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

import sys
sys.path.append('waveglow-mirror/')
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence
from denoiser import Denoiser

In [ ]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none')

#### Setup hparams

In [ ]:
hparams = create_hparams()
hparams.sampling_rate = 22050

#### Load model from checkpoint

In [ ]:
checkpoint_path = "tacotron2_statedict.pt"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path,map_location='cpu')['state_dict'])
_ = model.cpu().eval()

#### Load WaveGlow for mel2audio synthesis and denoiser

In [ ]:
waveglow_path = 'waveglow_256channels.pt'
waveglow = torch.load(waveglow_path,map_location='cpu')['model']
waveglow.cpu().eval()
for k in waveglow.convinv:
    k.float()
    
for m in waveglow.modules():
    if 'Conv' in str(type(m)):
        setattr(m, 'padding_mode', 'zeros')
        #print(m)

#denoiser = Denoiser(waveglow)

#### Prepare text input

In [ ]:
import time
start = time.time()

text = "Hi Paul! Waveglow is really awesome!"
sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cpu().long()

mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)

with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)

end = time.time()
print("Total calculation time %.2lf" % (end - start))

In [ ]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [ ]:
len(audio[0])/hparams.sampling_rate

In [ ]:
def generate_texts(texts):
  audios = None
  
  for text in texts:
    start = time.time()
    print("Calculating %s" % (text[:10]))
    sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
    sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cpu().long()

    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)

    with torch.no_grad():
        audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
    end = time.time()
        
    if audios is not None:
      audios = np.append(audios, audio[0].data.cpu().numpy())
    else:
      audios = audio[0].data.cpu().numpy()

    
    print("Total calculation time %.2lf, Length %.2lf, Real-Time: %.2lf" % (end - start, len(audio[0])/hparams.sampling_rate, (end - start)/(len(audio[0])/hparams.sampling_rate)))
      
  return audios 

In [ ]:
texts = ["May I have your attention please?", "May I have your attention please?", "Will the real Slim Shady please stand up?", "I repeat, will the real Slim Shady please stand up?", "We're gonna have a problem here."] + ["Cut my life into pieces.",
"This is my last resort.",
"Suffocation.",
"No breathing.",
"Don't give a fuck if I cut my arm bleeding.",
"This is my last resort."]
audios = generate_texts(texts)
ipd.Audio(audios, rate=hparams.sampling_rate)

In [ ]:
torch.set_num_threads(2)

In [ ]:
texts = ["May I have your attention please?", "May I have your attention please?", "Will the real Slim Shady please stand up?", "I repeat, will the real Slim Shady please stand up?", "We're gonna have a problem here."] + ["Cut my life into pieces.",
"This is my last resort.",
"Suffocation.",
"No breathing.",
"Don't give a fuck if I cut my arm bleeding.",
"This is my last resort."]
audios = generate_texts(texts)
ipd.Audio(audios, rate=hparams.sampling_rate)

In [ ]:
text = "Waveglow is really awesome!"
sequence = np.array(text_to_sequence(text, ['english_cleaners']))[None, :]
sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cpu().long()

#### Decode text input and plot results

In [ ]:
mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))

#### Synthesize audio from spectrogram using WaveGlow

In [ ]:
with torch.no_grad():
    audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

#### (Optional) Remove WaveGlow bias

In [ ]:
audio_denoised = denoiser(audio, strength=0.01)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate) 